In [ ]:
library(GEOquery); library(limma); library(samr); library(enrichR); library(tidyverse); library(fpc); library(cluster); library(EnhancedVolcano)
#Loading in glioblastoma microarray expression set
gset <- getGEO("GSE90598", GSEMatrix =TRUE, getGPL=FALSE)
if (length(gset) > 1) idx <- grep("GPL17692", attr(gset, "names")) else idx <- 1
gset <- gset[[idx]]

#Set expression set to a variable
ex <- exprs(gset)
# log2 transform
qx <- as.numeric(quantile(ex, c(0., 0.25, 0.5, 0.75, 0.99, 1.0), na.rm=T))
LogC <- (qx[5] > 100) ||
          (qx[6]-qx[1] > 50 && qx[2] > 0)
if (LogC) { ex[which(ex <= 0)] <- NaN
  ex <- log2(ex) }

In [ ]:
#Set column names to a more readable format
colnames(ex) <- pData(gset)[,1]
sampleinfo <- colnames(ex)
#Set as factor
sampleinfo$status = factor(ifelse(startsWith(sampleinfo,"Glioblastoma"),"glio","control"))
sampleinfo = as.data.frame(sampleinfo)
col.data <- sampleinfo

#Classes to colors for plotting
classes = sampleinfo$status
cols=rainbow(length(unique(classes)))
point.cols=cols[as.numeric(as.factor(classes))]

#Calculate sample variance for each gene
var.expr <- apply(ex,1,var)
#300 genes with highest sample variance
select=order(var.expr,decreasing=T)[1:300]
highvar.expr <- ex[select,]

#correlation distance
corr.dist=function(x) { as.dist(1-cor(t(x))) }

In [ ]:
#How samples cluster based on correlation distance, ward method
hc.ward.p <- hclust(corr.dist(t(highvar.expr)),method="ward.D2")
plot(hc.ward.p)

In [ ]:
#Principal component analysis
pc.expr <- prcomp(highvar.expr)
plot(pc.expr)

In [ ]:
#PCA, components 1 and 2, colored by case and control
plot(pc.expr$x[,1:2],col=point.cols)
legend("topright",inset = c(0,0),legend = unique(classes), fill = cols, cex = 0.7, xpd = TRUE)

In [ ]:
#Silhouette analysis for optimal number of clusters
num.clust <- c(2:10)
sil.widths <- vector()
mean.sil.width <- numeric()
for (i in num.clust) {
kmeans.result <- kmeans(highvar.expr,i)
sil.widths <- silhouette(kmeans.result$cluster, dist(highvar.expr))
mean.sil.width[i] <- mean(sil.widths[,3])
}
plot(num.clust,na.omit(mean.sil.width),xlab="Number of clusters",ylab="Average Silhouette Width")

In [ ]:
#Setting condition - if it is a glioblastoma sample =glio, if not=control
conditions <- as.factor(sampleinfo$status)

#Limma analysis of differential expression
design <- model.matrix(~ conditions + 0, gset)
#Consistent naming
colnames(design) <- levels(conditions)
rownames(design) <- colnames(ex)
#Set fit
fit <- lmFit(gset, design)
#Contrast matrix
cont.matrix <- makeContrasts(glio - control, levels=design)
fit2 <- contrasts.fit(fit, cont.matrix)
#Stats
fit2 <- eBayes(fit2, 0.01)
#Top table, adjusted with fdr. Arbitrarily high number
result.tt <- topTable(fit2, adjust="fdr", sort.by="B", number=25000000)

#Up-regulated genes
select.de.up = ! is.na(result.tt$P.Value) & result.tt$P.Value <= 0.05 & result.tt$logFC >=1
genes.de.up <- result.tt[select.de.up,]
print(paste("The number of upregulated genes is", dim(genes.de.up)[1]))
#1447 genes

In [ ]:
#Down-regulated genes
select.de.down = ! is.na(result.tt$P.Value) & result.tt$P.Value <= 0.05 & result.tt$logFC <=(-1)
genes.de.down <- result.tt[select.de.down,]
print(paste("The number of downregulated genes is", dim(genes.de.down)[1]))
#947 genes

In [ ]:
#SAM statistics, for comparison to limma/bayesian above
samfit <- SAM(x=ex,y=conditions,resp.type="Two class unpaired",geneid = rownames(ex))

In [ ]:
#Top 6 up-regulated
head(samfit[["siggenes.table"]][["genes.up"]])

In [ ]:
#Top 6 down-regulated
head(samfit[["siggenes.table"]][["genes.lo"]])

In [ ]:
## Annotate microarray probes
#Download file from publication
temp.labeldat <- read.delim("~/Downloads/GPL17692.an.txt")
temp.labeldat <- temp.labeldat[,1:2]
#Add new column of probe name IDs
result.tt$ProbeName <- as.numeric(row.names(result.tt))
#Join label data and limma top table results
temp.combin <- merge(temp.labeldat,result.tt,by="ProbeName")
#Remove rows with empty spaces
temp.combin.df <- temp.combin[!temp.combin$GeneSymbols=="", ]
#Some rows have multiple values in GeneSymbols, so expand the column and duplicate values
fin.combin.df <- temp.combin.df %>% separate_rows(GeneSymbols)

#Our top 300 genes of interest, ProbeName
sel.gene <- rownames(highvar.expr)
#Isolate 300 highest variance genes of interest
gene.enrich.dat <- fin.combin.df[which(fin.combin.df$ProbeName %in% sel.gene),]
#I can only get 267 genes due to data missingness

#Using the new, gene selected DE results, isolate upregulated and downregulated
#Upregulated
sel.de.up = ! is.na(gene.enrich.dat$P.Value) & gene.enrich.dat$P.Value <= 0.05 & gene.enrich.dat$logFC >=1
sel.genes.de.up <- gene.enrich.dat[sel.de.up,]
print(paste("The number of upregulated genes is", dim(sel.genes.de.up)[1]))
#59 genes
#Downregulated
sel.de.down = ! is.na(gene.enrich.dat$P.Value) & gene.enrich.dat$P.Value <= 0.05 & gene.enrich.dat$logFC <=(-1)
sel.genes.de.down <- gene.enrich.dat[sel.de.down,]
print(paste("The number of downregulated genes is", dim(sel.genes.de.down)[1]))
#132 genes

In [ ]:
#Set databases of interest for enrichment analysis via enrichR
dbs <- c("GO_Molecular_Function_2018", "GO_Cellular_Component_2018", "GO_Biological_Process_2018","KEGG_2016","Reactome_2016")
#For all differentially expressed genes
gene.enrich.list <- enrichr(gene.enrich.dat$GeneSymbols,dbs)

In [ ]:
#Plotting enrichment analysis for molecular functions
plotEnrich(gene.enrich.list[[1]], showTerms = 20, numChar = 40, y = "Count", orderBy = "P.value")

In [ ]:
#Plotting enrichment analysis for biological processes
plotEnrich(gene.enrich.list[[3]], showTerms = 20, numChar = 40, y = "Count", orderBy = "P.value")

In [ ]:
#Plotting KEGG enrichment analysis
plotEnrich(gene.enrich.list[[4]], showTerms = 20, numChar = 40, y = "Count", orderBy = "P.value")

In [ ]:
#Simple Volcano Plot
EnhancedVolcano(gene.enrich.dat,
                 lab = gene.enrich.dat$GeneSymbols,
                 x = 'logFC',
                pCutoff = 10e-5,
                FCcutoff = 1.5,
                 y = 'P.Value')

In [ ]:
#Repeat above, For upregulated differentially expressed genes
up.gene.enrich.list <- enrichr(sel.genes.de.up$GeneSymbols,dbs)

In [ ]:
#Plotting KEGG enrichment analysis
plotEnrich(up.gene.enrich.list[[4]], showTerms = 20, numChar = 40, y = "Count", orderBy = "P.value

In [ ]:
#Repeat above, For downregulated differentially expressed genes
down.gene.enrich.list <- enrichr(sel.genes.de.down$GeneSymbols,dbs)

In [ ]:
#Plotting KEGG enrichment analysis
plotEnrich(down.gene.enrich.list[[4]], showTerms = 20, numChar = 40, y = "Count", orderBy = "P.value")

In [ ]:
#Setting up network analysis, interactome analysis cluster
IAC=cor(t(highvar.expr),use="p")
hist(IAC,sub=paste("Mean=",format(mean(IAC[upper.tri(IAC)]),digits=3)))

In [ ]:
#Prep data for WGCNA
cluster1=hclust(as.dist(1-IAC))
keepGenesExpr = rank(-rowMeans(ex))<=1000
filt.expr<-ex[keepGenesExpr,]
dataExpr <- filt.expr[c(1:1000),]
dataExpr <- t(dataExpr)

In [ ]:
library(WGCNA)
net = blockwiseModules(dataExpr, power = 7,
                       TOMType = "signed", minModuleSize = 30,
                       reassignThreshold =10, mergeCutHeight = 0.5,
                       numericLabels = TRUE, pamRespectsDendro = FALSE,
                       saveTOMs = TRUE,
                       saveTOMFileBase="TOM", verbose=3, ds=3)

In [ ]:
#Generate cluster dendrogram
mergedColors = labels2colors(net$colors)
plotDendroAndColors(net$dendrograms[[1]], mergedColors[net$blockGenes[[1]]],
"Module colors",
dendroLabels = FALSE, hang = 0.03,
addGuide = TRUE, guideHang = 0.05)

In [ ]:
#Prep for import to Cytoscape
genes=colnames(dataExpr)
moduleColors=labels2colors(net$colors)
mymodules=cbind(genes,moduleColors)

load("TOM-block.1.RData")
testing <- as.matrix(TOM)
dimnames(testing) <- list(genes,genes)

suppressMessages(exportNetworkToCytoscape(testing, edgeFile="edgedata.txt", nodeFile="nodedata.txt", weighted=TRUE, threshold = 0.1, nodeNames = genes, nodeAttr = moduleColors))

In [ ]:
#STRING
library(STRINGdb)
#Stringdb expects a dataframe
fix.str.dat <- as.data.frame(gene.enrich.dat)
#Setup. Threshold of 0 to capture all interactions
string_db = STRINGdb$new(version="11.5",species=9606, score_threshold=0,input_directory="")
DE.mapped = string_db$map(fix.str.dat,"GeneSymbols",removeUnmappedRows = TRUE)

In [ ]:
#14% of genes couldn't get mapped
#100 genes with the most significant DE (out of genes we could map):
top.genes = DE.mapped$STRING_id[order(DE.mapped$P.Value)[1:100]]
#generate the PPI network with all the interactions among the specified genes:
string_db$plot_network(top.genes)

In [ ]:
# filter by p-value and add a color column
# green down-regulated genes and red for up-regulated genes
topgenes_pval05 <- string_db$add_diff_exp_color( subset(DE.mapped, P.Value<0.05),logFcColStr="logFC" )
# post payload information to the STRING server
payload_id <- string_db$post_payload( topgenes_pval05$STRING_id,colors=topgenes_pval05$color )
# display a STRING network png with the "halo"
string_db$plot_network( top.genes, payload_id=payload_id )

In [ ]:
#Generate enrichment data
enrichdat <- string_db$get_enrichment(top.genes)
head(enrichdat)